# Figure Materials

Filters etc. for use in explanatory and result figures.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

import torch
import torch.nn.functional as F
from torchvision import models

# work from project root for local imports
import os
import sys
import subprocess
from pathlib import Path

# root here refers to the segmentron-master folder
root_dir = Path(subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode("utf-8"))
root_dir = root_dir / "segmentron-master"
os.chdir(root_dir)
sys.path.append(str(root_dir))

from sigma.blur import blur2d_sphere, blur2d_diag, blur2d_full
from sigma.blur import gauss2d_full
from sigma.blur import conv2d_mono
from sigma.blur import logchol2sigma, sigma2logchol

torch.manual_seed(1337)

# Compositional Filtering Figure

Load test images of impulse and Gauss himself.

In [ ]:
im_pulse = torch.zeros(11, 11)
im_pulse.view(-1)[im_pulse.numel() // 2] = 1.

im_gauss = torch.tensor(np.array(Image.open('notebooks/gauss.jpg'))).float()
im_gauss = (im_gauss - im_gauss.min()) / (im_gauss.max() - im_gauss.min())
im_gauss = im_gauss[:, 1:]

ims = (im_pulse, im_gauss)

plt.figure(figsize=(5 * len(ims), 5))
for i, im in enumerate(ims, 1):
    plt.subplot(1, len(ims), i)
    plt.imshow(im.numpy())

Load pretrained weights.

In [ ]:
alexnet = models.alexnet(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

Visualize first layer filters of ResNet-50 or AlexNet.

In [ ]:
#conv1 = alexnet.state_dict()['features.0.weight']
conv1 = resnet50.state_dict()['conv1.weight']

filters = conv1.numpy().transpose(0, 2, 3, 1)
filters -= filters.min()
filters /= filters.max()

plt.figure(figsize=(10, 10))
for i in range(8):
    for j in range(8):
        idx = i*8 + j
        plt.subplot(8, 8, idx + 1)
        plt.imshow(filters[idx])
        plt.axis('off')

Let's simplify to monochrome by averaging out color, have another look, and then pick a few favorites as `free_form`.

In [ ]:
conv1_grey = conv1.mean(1)

plt.figure(figsize=(16, 16))
for i, f in enumerate(conv1_grey):
    plt.subplot(8, 8, i + 1)
    #plt.title(f"#{i}")
    plt.imshow(f.numpy())
    plt.axis('off')
plt.tight_layout()

free_form_idx = [0, 1, 5, 15, 33]
free_form = conv1_grey[free_form_idx]

plt.figure()
for i, f in enumerate(free_form):
    plt.subplot(1, len(free_form), i + 1)
    plt.imshow(f.numpy())
    plt.axis('off')
plt.tight_layout()

Instantiate a variety of Gaussians and pick our favorites as `gaussian` with covariance parameters `cov`.

In [ ]:
std_devs = 2  # 95% coverage

cov_delta = torch.tensor([-1., 0., -1.])
cov_standard = torch.tensor([0.1, 0., 0.1])
cov_sphere = torch.tensor([0.7, 0., 0.7])
cov_diag = torch.tensor([0.5, 0., -0.5])
cov_full = torch.tensor([0.2, 1, 0.2])
cov_rand = torch.randn(64, 3) * 0.5

cov_struct = torch.stack([cov_delta, cov_standard, cov_sphere, cov_diag, cov_full])

g_struct = gauss2d_full(cov_struct, std_devs=std_devs)
g_rand = gauss2d_full(cov_rand, std_devs=std_devs)

gs = g_struct
plt.figure(figsize=(2 * len(gs), 2))
for i, g in enumerate(gs, 1):
    plt.subplot(1, len(gs), i)
    plt.imshow(g, aspect='equal')
    plt.axis('off')
plt.tight_layout()
#plt.savefig('compose-gaussians.png', bbox_inches='tight', pad_inches=0)

gs = g_rand
plt.figure(figsize=(16, 16))
for i, f in enumerate(g_rand):
    plt.subplot(8, 8, i + 1)
    #plt.title(f"#{i}")
    plt.imshow(f[2:-2, 2:-2].numpy())
    plt.axis('off')
plt.tight_layout()

gaussian = g_struct
cov = cov_struct

At heart filter composition is simply convolution, but for visualization we need to rig up some tooling.

In [ ]:
def compose_filters(f, g, dil=None, normalize_f=False):
    if dil is None:
        dil = torch.ones(2).int()
        
    if normalize_f:
        f -= f.mean()
        f /= f.max()
        
    f_imp = filter_impulse(f, dil=dil)
    
    g_ks = torch.tensor(g.size()[2:]).int()
    gof = F.conv2d(f_imp, g, padding=list(g_ks // 2))
    return gof


def filter_impulse(f, dil=None):
    if dil is None:
        dil = torch.ones(2).int()
        
    f_ks = torch.tensor(f.size()[2:]).int()
    eff_ks = f_ks + (f_ks - 1) * (dil - 1)
    
    impulse = torch.zeros(1, 1, eff_ks[0], eff_ks[1])
    impulse.view(-1)[impulse.numel() // 2] = 1.
    f_imp = F.conv2d(impulse, f, padding=list((eff_ks + dil) // 2), dilation=list(dil))
    f_imp = f_imp.flip((2, 3))
    return f_imp


def fit_to_max_size(arrs):
    max_size, _ = torch.stack([torch.tensor(arr.size()[2:]) for arr in arrs]).max(0)
    for i, arr in enumerate(arrs):
        pad_h, pad_w = (max_size - torch.tensor(arr.size()[2:])) // 2
        arrs[i] = F.pad(arr, (pad_w, pad_w, pad_h, pad_h))

    return arrs

    
def normalize_minmax(arr):
    arr = (arr - arr.min()) / (arr.max() - arr.min())
    return arr
    
    
def normalize_meanmax(arr):
    arr = (arr - arr.mean()) / (arr.max() - arr.mean())
    return arr

Free-forms, Gaussians, Composition!

In [ ]:
f_idx = 1
g_idx = 2

f = free_form[f_idx].clone()
g = gaussian[g_idx].clone()
c = cov[g_idx].clone()

c_ = c * 0.5
g = gauss2d_full(c_)[0]

# normalize f
f = normalize_meanmax(f)

# switch to pytorch filter dims:
# output x input x h x w
f = f.view(1, 1, *f.size())
g = g.view(1, 1, *g.size())

sigma = c_[[0, -1]].exp()
sigmat = logchol2sigma(c_)

#dil = torch.ones(2).int()
dil = (sigma * 2).round().clamp(min=1.).int()

imp = filter_impulse(f, dil=dil)
gof = compose_filters(f, g, dil=dil)
gof = gof[..., 1:-1, 1:-1]

In [ ]:
# normalize separately, as done automatically by plotting
f_, g_, gof_ = f, g, gof

print(gof_.size())
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title(f'free-form #{f_idx}')
plt.imshow(f_.squeeze().numpy())
plt.subplot(1, 3, 2)
plt.title(f'gaussian #{g_idx} $\Sigma = $ [{sigmat[0, 0]:.2f}, {sigmat[0, 1]:.2f}, {sigmat[1, 1]:.2f}]')
plt.imshow(g_.squeeze().numpy())
plt.subplot(1, 3, 3)
plt.title('composition')
plt.imshow(gof_.squeeze().numpy())

In [ ]:
# fit to same size
# normalize by mean/max
f_, g_, gof_ = f, g, gof
f_, g_, gof_ = fit_to_max_size([f_, g_, gof_])
f_, gof_ = [normalize_meanmax(n) for n in [f_, gof_]]

min_all, _ = torch.cat((f_.view(-1), g_.view(-1), gof_.view(-1))).min(0)
max_all, _ = torch.cat((f_.view(-1), g_.view(-1), gof_.view(-1))).max(0)

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title(f'free-form #{f_idx}')
plt.imshow(f_.squeeze().numpy())
plt.subplot(1, 3, 2)
plt.title(f'gaussian #{g_idx} $\Sigma = $ [{sigmat[0, 0]:.2f}, {sigmat[0, 1]:.2f}, {sigmat[1, 1]:.2f}]')
plt.imshow(g_.squeeze().numpy())
plt.subplot(1, 3, 3)
plt.title('composition')
plt.imshow(gof_.squeeze().numpy(), vmin=min_all, vmax=max_all)

In [ ]:
# fit to same size, 
# normalize by mean/max
# combine
def show_fit_norm_combine(f, g, gof):
    f_, g_, gof_ = f, g, gof
    f_, g_, gof_ = fit_to_max_size([f, g, gof])
    f_, g_, gof_ = [normalize_meanmax(n) for n in [f_, g_, gof_]]

    combined = torch.cat((f_, g_, gof_), -1)
    plt.imshow(combined.squeeze().numpy())
    plt.axis('off')

plt.figure(figsize=(15, 5))
show_fit_norm_combine(f, g, gof)

Let's inspect the impulse to see the effect of dilation.

In [ ]:
dils = torch.tensor([1, 1, 2, 1, 2, 2, 3, 3]).view(-1, 2).int()

imps = [filter_impulse(f, dil=dil) for dil in dils]
imps = fit_to_max_size(imps)

plt.figure(figsize=(5, 5))
plt.title(f'filter #{f_idx}')
plt.imshow(f.squeeze().numpy())
plt.axis('off')

plt.figure(figsize=(4 * dils.size(0), 4))
for i, (imp, dil) in enumerate(zip(imps, dils), 1):
    plt.subplot(1, dils.size(0), i)
    plt.title(f'impulse dilation {(dil[0].item(), dil[1].item())}')
    plt.imshow(imp.squeeze().numpy())
    plt.axis('off')

Plot composition of all the free-form filters x gaussians:

1. with blur alone
1. with blur + dilation where dilation rate = $2\sigma$

The first enlarges and blurs the filters, while the second purely enlarges.

In [ ]:
fs, gs, gofs = [], [], []
for f_idx in range(len(free_form)):
    for g_idx in range(len(gaussian)):
        f = free_form[f_idx].clone()
        g = gaussian[g_idx].clone()
        c = cov[g_idx].clone()

        # normalize f
        f = normalize_meanmax(f)

        # switch to pytorch filter dims:
        # output x input x h x w
        f = f.view(1, 1, *f.size())
        g = g.view(1, 1, *g.size())

        sigma = c[[0, -1]].exp()
        sigmat = logchol2sigma(c)

        dil = torch.ones(2).int()  # n.b. constant

        imp = filter_impulse(f, dil=dil)
        gof = compose_filters(f, g, dil=dil)
        
        #f, g, gof = fit_to_max_size([f, g, gof])
        f, gof = [normalize_meanmax(n) for n in [f, gof]]
        
        fs.append(f)
        gs.append(g)
        gofs.append(gof)
        
plt.figure(figsize=(10, 85))
for i in range(0, len(free_form) * len(gaussian) * 3, 3):
    plt.subplot(len(free_form) * len(gaussian), 3, i + 1)
    plt.imshow(fs[i // 3].squeeze().numpy(), aspect='equal')
    plt.axis('off')
    plt.subplot(len(free_form) * len(gaussian), 3, i + 2)
    plt.imshow(gs[i // 3].squeeze().numpy(), aspect='equal')
    plt.axis('off')
    plt.subplot(len(free_form) * len(gaussian), 3, i + 3)
    #plt.title(tuple(gofs[i // 3].size()[-2:]))
    plt.imshow(gofs[i // 3].squeeze().numpy(), aspect='equal')
    plt.axis('off')
plt.subplots_adjust(wspace=0.1, hspace=0.1, bottom=0, left=0, top=1, right=1)
#plt.savefig('compose-blur-only-all.png', bbox_inches='tight', pad_inches=0)

In [ ]:
fs, gs, gofs = [], [], []
for f_idx in range(len(free_form)):
    for g_idx in range(len(gaussian)):
        f = free_form[f_idx].clone()
        g = gaussian[g_idx].clone()
        c = cov[g_idx].clone()

        # normalize f
        f = normalize_meanmax(f)

        # switch to pytorch filter dims:
        # output x input x h x w
        f = f.view(1, 1, *f.size())
        g = g.view(1, 1, *g.size())

        sigma = c[[0, -1]].exp()
        sigmat = logchol2sigma(c)

        dil = (sigma * 2).round().clamp(min=1.).int()  # n.b. function of sigma

        imp = filter_impulse(f, dil=dil)
        gof = compose_filters(f, g, dil=dil)
        
        #f, g, gof = fit_to_max_size([f, g, gof])
        f, gof = [normalize_meanmax(n) for n in [f, gof]]
        
        fs.append(f)
        gs.append(g)
        gofs.append(gof)
        
plt.figure(figsize=(10, 85))
for i in range(0, len(free_form) * len(gaussian) * 3, 3):
    plt.subplot(len(free_form) * len(gaussian), 3, i + 1)
    plt.imshow(fs[i // 3].squeeze().numpy(), aspect='equal')
    plt.axis('off')
    plt.subplot(len(free_form) * len(gaussian), 3, i + 2)
    plt.imshow(gs[i // 3].squeeze().numpy(), aspect='equal')
    plt.axis('off')
    plt.subplot(len(free_form) * len(gaussian), 3, i + 3)
    plt.title(tuple(gofs[i // 3].size()[-2:]))
    plt.imshow(gofs[i // 3].squeeze().numpy(), aspect='equal')
    plt.axis('off')
plt.subplots_adjust(wspace=0.1, hspace=0.1, bottom=0, left=0, top=1, right=1)
#plt.savefig('compose-blur-dilate-all.png', bbox_inches='tight', pad_inches=0)

In [ ]:
fs, gs, gofs = [], [], []
for idx in range(len(free_form)):
    f_idx = idx
    g_idx = idx
    f = free_form[f_idx].clone()
    g = gaussian[g_idx].clone()
    c = cov[g_idx].clone()

    # normalize f
    f = normalize_meanmax(f)

    # switch to pytorch filter dims:
    # output x input x h x w
    f = f.view(1, 1, *f.size())
    g = g.view(1, 1, *g.size())

    sigma = c[[0, -1]].exp()
    sigmat = logchol2sigma(c)

    dil = (sigma * 2).round().clamp(min=1.).int()

    imp = filter_impulse(f, dil=dil)
    gof = compose_filters(f, g, dil=dil)

    f, gof = fit_to_max_size([f, gof])
    f, g, gof = normalize_minmax(f), normalize_minmax(g), normalize_minmax(gof)

    fs.append(f)
    gs.append(g)
    gofs.append(gof)
        
fs, gs, gofs = fit_to_max_size(fs), gs, fit_to_max_size(gofs)
#f, g, gof = normalize_filters_minmax(f, g, gof)

plt.figure(figsize=(30, 30))
for i in range(len(free_form)):
    plt.subplot(len(free_form), 3, i*3 + 1)
    plt.imshow(fs[i].squeeze().numpy(), aspect='equal')
    plt.axis('off')
    plt.subplot(len(free_form), 3, i*3 + 2)
    plt.imshow(gs[i].squeeze().numpy(), aspect='equal')
    plt.axis('off')
    plt.subplot(len(free_form), 3, i*3 + 3)
    plt.imshow(gofs[i].squeeze().numpy(), aspect='equal')
    plt.axis('off')
#plt.savefig('compose-blur-only-each.png', bbox_inches='tight', pad_inches=0)

# Smooth + Circular Dilation

First, let's illustrate how dilation can violate the sampling theorem and cause aliasing.

In [ ]:
dil = torch.tensor((8,) * 2)
half_dil = True

for im in ims:
    ks = torch.tensor((3,) * 2)
    if half_dil:
        dil = torch.tensor(im.size()) // 2
    eff_ks = ks + (ks - 1) * (dil - 1)
    pad = eff_ks // 2 + 1
    
    im = im.view(1, 1, *im.size())
    w = torch.ones(1, 1, *ks)
    out = F.conv2d(im, w, dilation=tuple(dil), padding=tuple(pad))
    
    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(im.squeeze().numpy())
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(out.squeeze().numpy())
    plt.axis('off')